# Lab 2: Numba

Links: 
- https://github.com/jpivarski-talks/2021-02-03-pyhep-numba-tutorial
- https://henryiii.github.io/compclass/week10/2_rk.html
- https://carpentries-incubator.github.io/lesson-gpu-programming/02-cupy/index.html
- https://docs.google.com/presentation/d/13MmOKh30IbRJo5sZ8ZujQFr1J3rhqDjA-0Qkfg7JZmE/edit#slide=id.p
- https://indico.cern.ch/event/1019958/timetable/#43-cuda-and-python-with-numba

## Simple stuff

Even if you've used Numba before, let's start with the basics.

In fact, let's start with NumPy itself.

### NumPy

In [ ]:
import numpy as np

NumPy accelerates Python code by replacing loops in Python's virtual machine (with type-checks at runtime) with precompiled loops that transform arrays into arrays.

In [ ]:
data = np.arange(1000000)
data

In [ ]:
%%timeit

output = np.empty(len(data))

for i, x in enumerate(data):
    output[i] = x**2

In [ ]:
%%timeit

output = data**2

But if you have to compute a complex formula, NumPy would have to _make an array for each intermediate step_.

(There are tricks for circumventing this, but we won't get into that.)

In [ ]:
energy = np.random.normal(100, 10, 1000000)
px = np.random.normal(0, 10, 1000000)
py = np.random.normal(0, 10, 1000000)
pz = np.random.normal(0, 10, 1000000)

In [ ]:
%%timeit

mass = np.sqrt(energy**2 - px**2 - py**2 - pz**2)

The above is equivalent to

In [ ]:
%%timeit

tmp1 = energy**2
tmp2 = px**2
tmp3 = py**2
tmp4 = pz**2
tmp5 = tmp1 - tmp2
tmp6 = tmp5 - tmp3
tmp7 = tmp6 - tmp4
mass = np.sqrt(tmp7)

### Numba

In [ ]:
import numba as nb

Numba lets us compile a function to compute a whole formula in one step.

```python
@nb.jit
```

means "JIT-compile" and

```python
@nb.njit
```

means "really JIT-compile" because the original has a fallback mode that's getting deprecated. If we're using Numba at all, we don't want it to fall back to ordinary Python.

In [ ]:
@nb.njit
def compute_mass(energy, px, py, pz):
    mass = np.empty(len(energy))
    for i in range(len(energy)):
        mass[i] = np.sqrt(energy[i]**2 - px[i]**2 - py[i]**2 - pz[i]**2)
    return mass

The `compute_mass` function is now "ready to be compiled." It will be compiled when we give it arguments, so that it can propagate types.

In [ ]:
compute_mass(energy, px, py, pz)

In [ ]:
%%timeit

mass = compute_mass(energy, px, py, pz)

## Type considerations

### Dynamically typed programs that are not statically typed

Much of Python's flexibility comes from the fact that it does not need to know the types of all variables before it starts to run.

That dynamism makes it easier to express complex logic (what I call "bookkeeping"), but it is a hurdle for speed.

### Avoid lists and dicts

Although Numba developers are doing a lot of work on supporting Python `list` and `dict` (of identically typed contents), I find it to be too easy to run into unsupported cases. The main problem is that their contents _must_ be typed, but the Python language didn't have ways to express types.

(Yes, Python has type annotations now, but they're not fully integrated into Numba yet.)

## Runge–Kutta example

Let's use Numba to accelerate RK4


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x_max = 1  # Size of x max
v_0 = 0
koverm = 1  # k / m


def f(t, y):
    "Y has two elements, x and v"
    return np.array([-koverm * y[1], y[0]])

In [ ]:
def rk4_ivp(f, init_y, t):
    steps = len(t)
    order = len(init_y)

    y = np.empty((steps, order))
    y[0] = init_y

    for n in range(steps - 1):
        h = t[n + 1] - t[n]

        k1 = h * f(t[n], y[n])  # 2.1
        k2 = h * f(t[n] + h / 2, y[n] + k1 / 2)  # 2.2
        k3 = h * f(t[n] + h / 2, y[n] + k2 / 2)  # 2.3
        k4 = h * f(t[n] + h, y[n] + k3)  # 2.4

        y[n + 1] = y[n] + 1 / 6 * (k1 + 2 * k2 + 2 * k3 + k4)  # 2.0

    return y

In [ ]:
ts = np.linspace(0, 40, 100 + 1)
y = rk4_ivp(f, [x_max, v_0], ts)
plt.plot(ts, np.cos(ts))
plt.plot(ts, y[:, 0], "--")

In [ ]:
%%timeit
ts = np.linspace(0, 40, 1000 + 1)
y = rk4_ivp(f, [x_max, v_0], ts)